In [1]:
import os
import xml.etree.ElementTree as ET 
import struct
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.core.debugger import Tracer

In [12]:
ROOT_DIR = "doodle_detection"

In [13]:
def parse_xml(image_dir, xml_file):
#     image_dir = ""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_paths = []
    labels = []

    images = root.find("images")
    images = images.findall("image")
    print("Number of images to parse: " + str(len(images)))

    for idx, image in enumerate(images):
        if idx % 100 == 0:
            print(idx)
        image_name = image.get('file').split('\\')[-1]
        image_path = os.path.join(image_dir, image_name)
        print(image_path)
        if not os.path.exists(image_path):
            print("wrong img name: " + xml_file)
            continue
        cur_img = cv2.imread(image_path)
#         if cur_img == None:
#             print("wrong img name: " + xml_file)
#             continue
        img_height, img_width, _ = cur_img.shape

        image_labels = []
        boxes = image.findall('box')
        if len(boxes) == 0:
            print("no label data: " + xml_file)
            continue
        for box in boxes:
#             obj_label = box.find('label')
#             if obj_label.text == "bar":
#                 continue
            top = int(box.get('top'))
            left = int(box.get('left'))
            width = int(box.get('width'))
            height = int(box.get('height'))

            xmin = np.max([left, 0])
            xmax = np.min([left + width, img_width])
            ymin = np.max([top, 0])
            ymax = np.min([top + height, img_height])

            image_labels.append([xmin, ymin, xmax, ymax, 0])
        
        if len(image_labels) == 0:
            continue

        image_paths.append(os.path.join(image_dir, image_name))
        labels.append(image_labels)
    return image_paths, labels

In [14]:
image_paths = []
labels = []
anims_root_dir = os.listdir(ROOT_DIR)
for anim in anims_root_dir:
    anim_path = os.path.join(ROOT_DIR, anim)
    DATA_DIRS = os.listdir(anim_path)
    DATA_DIRS = [e for e in DATA_DIRS if e.endswith('.xml')]
    for data_dir in DATA_DIRS:
        img_name = data_dir.split(".")[0]
        image_dir = os.path.join(anim_path,img_name)
        if os.path.exists(image_dir) == False:
            continue

        xml_file = os.path.join(anim_path, data_dir)
        print(xml_file)
        print(image_dir)
        cur_image_paths, cur_labels = parse_xml(image_dir, xml_file)
        image_paths += cur_image_paths
        labels += cur_labels

doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2
Number of images to parse: 50
0
doodle_detection/hedge_hog/Annotator_2/IMG_4973.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4974.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4975.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4976.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4977.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4978.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4979.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle_detection/hedge_hog/Annotator_2/IMG_4980.png
no label data: doodle_detection/hedge_hog/Annotator_2.xml
doodle

In [15]:
def convert_to_string(image_path, labels):
    """convert image_path, lables to string 
    Returns:
    string 
    """
    out_string = ''
    out_string += image_path
    for label in labels:
        for i in label:
            out_string += ' ' + str(i)
    out_string += '\n'
    return out_string

records = []
for idx, image_path in enumerate(image_paths):
    label = labels[idx]
    records.append(convert_to_string(image_path, label))

random.shuffle(records)

In [18]:
val_ratio = 0.1
total_num = len(records)
print(total_num)
val_num = int(val_ratio * total_num)
print("val: " + str(val_num))
train_num = total_num - val_num
print("train " + str(train_num))
train_records = records[0:train_num]
val_records = records[train_num:]

2523
val: 252
train 2271


In [19]:
train_out_file = open("draw_train.txt", "w")
for record in train_records:
    train_out_file.write(record)
train_out_file.close()
val_out_file = open("draw_val.txt", "w")
for record in val_records:
    val_out_file.write(record)
val_out_file.close()